<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_overlap_50_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [6]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0     50      0  0:00:05  0:00:05 --:--:--    68
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [7]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [13]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,purple sploches around waist
3,7,goiter
4,8,ARTHRITIS


In [14]:
# testing data set

test = pd.read_csv(path/"AskAPatient_overlap_50.csv",header=None)
test.head()

,0,1
0,0,'scared' feeling
1,2,heel pain
2,3,cracking sensations in my joints
3,4,purple sploches around waist
4,5,fast and slow heartbeat


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_cadec_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [19]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [23]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.849347,4.222500,0.216869,03:57


In [24]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.989234,3.713879,0.293495,03:57


In [25]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.709468,3.683853,0.298119,03:58


In [26]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.668819,3.661555,0.300255,03:54


In [0]:
learn.save_encoder('AskAPatient_overlap_50_first')

In [28]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('AskAPatient_overlap_50_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.952562,3.557249,0.401183,01:14


In [29]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.759012,2.982780,0.501775,01:30


In [30]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.402510,2.661853,0.559763,03:52


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.108040,2.378290,0.594083,03:56
1,1.751212,2.195771,0.630769,04:00


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.622818,2.003335,0.660355,03:49
1,1.411817,1.907140,0.671006,03:34


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.426548,1.791969,0.691124,03:31
1,1.227673,1.703487,0.702959,04:01


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.138234,1.654338,0.714793,03:52
1,1.035880,1.578298,0.733728,03:45


In [35]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.973039,1.571251,0.737278,03:50
1,0.927239,1.479206,0.745562,03:28


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.847102,1.537669,0.744379,03:47
1,0.882296,1.467405,0.750296,03:53
2,0.773315,1.413571,0.772781,03:36
3,0.684253,1.418788,0.772781,03:40


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.667764,1.461529,0.764497,03:42
1,0.764356,1.428011,0.770414,03:46
2,0.659974,1.389306,0.777515,03:33
3,0.604518,1.395294,0.779882,03:34


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.609861,1.442950,0.771598,03:29
1,0.632271,1.522272,0.771598,03:53
2,0.563899,1.383939,0.789349,03:27
3,0.540976,1.430104,0.783432,03:30


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.527094,1.447261,0.772781,03:37
1,0.567509,1.481953,0.775148,03:33
2,0.562092,1.402750,0.782248,03:39
3,0.473646,1.433579,0.783432,03:48


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.495471,1.452841,0.779882,03:36
1,0.535470,1.395574,0.779882,03:37
2,0.474589,1.398302,0.783432,03:48
3,0.435573,1.398490,0.785799,03:47


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.504858,1.477849,0.786982,03:31
1,0.515791,1.541177,0.792899,03:46
2,0.496942,1.502005,0.791716,03:34
3,0.444007,1.440294,0.794083,03:58


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.483692,1.474134,0.788166,03:47
1,0.469414,1.467598,0.784615,03:45
2,0.432906,1.458059,0.790533,03:48
3,0.462790,1.442362,0.790533,03:46


In [43]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.449780,1.463937,0.784615,03:34
1,0.519230,1.475873,0.788166,03:42
2,0.473085,1.478184,0.788166,03:24
3,0.429883,1.495082,0.790533,03:31


In [44]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.422661,1.532468,0.789349,03:51
1,0.491974,1.461807,0.789349,03:28
2,0.443358,1.534326,0.791716,03:34
3,0.441941,1.601632,0.786982,03:43


In [45]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.430156,1.534223,0.788166,03:53
1,0.473192,1.512355,0.789349,03:34
2,0.423902,1.465293,0.791716,03:42
3,0.422353,1.521817,0.792899,03:55


In [0]:
# save the best model

learn.save_encoder('AskAPatient_overlap_50')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [48]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

1221
1221
[103, 2, 910, 878, 81, 5, 194, 1031, 8, 8, 8, 8, 8, 8, 8, 126, 10, 123, 788, 71, 21, 1031, 610, 27, 27, 1028, 1003, 29, 614, 289, 30, 414, 414, 32, 34, 76, 289, 37, 37, 37, 37, 37, 289, 218, 640, 515, 655, 194, 1031, 1031, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 100, 100, 100, 100, 100, 764, 53, 53, 53, 58, 59, 61, 770, 71, 71, 71, 71, 491, 76, 76, 76, 260, 753, 122, 614, 710, 1031, 614, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 289, 289, 614, 614, 614, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 202, 88, 89, 268, 391, 202, 753, 92, 850, 94, 94, 94, 453, 94, 94, 94, 94, 94, 94, 761, 337, 533, 683, 98, 794, 337, 100, 100, 337, 100, 100, 100, 100, 58, 100, 100, 101, 337, 72, 660, 660, 660, 636, 1035, 313, 289, 289, 289, 260, 1035, 565, 126, 289, 289, 730, 117, 1031, 120, 135, 135, 753, 1031, 1031, 125, 778, 778, 778, 778, 778, 778, 778, 778, 125, 1031, 796, 127, 

In [49]:
correct_num = 0

for i in range(0, 1221):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

627
0.5135135135135135
